In [1]:
from CoRe_Dataloader_From_Files_With_OvGN import (
    get_new_train_validation_test_datasets,
    get_new_train_validation_test_dataloaders
)
from CoRe_Dataloader_ECSG import dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import math
import torchinfo
import time
import numpy as np
import wandb
import datetime
from collections import OrderedDict
from torch import autograd

[['BAM:0136', 'R04', 'rh_22', 'Rh_l2_m2_r00900.txt', 0], ['BAM:0037', 'R01', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['BAM:0027', 'R01', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0036', 'R01', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['THC:0019', 'R03', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['THC:0085', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0076', 'R01', 'rh_22', 'Rh_l2_m2_r00900.txt', 0], ['THC:0084', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0100', 'R29', 'rh_22', 'Rh_l2_m2_r00850.txt', 0], ['THC:0084', 'R01', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0087', 'R06', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['THC:0072', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['THC:0014', 'R01', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0057', 'R02', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['THC:0057', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['THC:0097', 'R01', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0123', 'R05', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['BAM:0141', 'R03', 'rh_22', 'Rh_l2_m2_r00900.t

In [2]:
import torchmetrics as metrics
import pandas as pd

mae = metrics.MeanAbsoluteError()
mse = metrics.MeanSquaredError()
combined = metrics.MetricCollection(
    [mae, mse, metrics.MeanAbsolutePercentageError(), metrics.MeanSquaredLogError()]
)


def get_df_from_rdict(rdict):
    return pd.DataFrame(pd.Series(rdict).map(lambda x: x.cpu().item())).T

In [3]:
# import vit
# import vit_pytorch
from vit_pytorch import vit_for_small_dataset as vit_sd
from vit_pytorch import vit as simple_vit
from vit_pytorch.deepvit import DeepViT


def init_model():
    # return simple_vit.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=int(1024/2),
    #                depth=2,
    #                heads=8,
    #                mlp_dim=int(2048/2),
    #                channels=1).to("cuda:0")
    # return vit_sd.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=1024,
    #                depth=4,
    #                heads=16,
    #                mlp_dim=int(2048/2),
    #                dropout = 0.1,
    #                emb_dropout = 0,
    #                channels=1).to("cuda:0")
    return DeepViT(
        image_size=400,
        patch_size=20,
        num_classes=2,
        dim=1024,
        depth=4,
        heads=16,
        mlp_dim=int(2048 / 2),
        dropout=0.1,
        emb_dropout=0.1,
        channels=1,
    ).to("cuda:0")

In [4]:
dumstring = " "
def calc_metrics(model: torch.nn.Module, dl: DataLoader):
    model.eval()
    raw_output = []
    parameters = []
    with torch.no_grad():
        for batch, (sg, params) in enumerate(dl):
            sg = sg.to("cuda:0").float()
            sgsh = sg.shape
            sg = sg.view(sgsh[0], 1, sgsh[1], sgsh[2])

            params = params[:, 1:3].to("cuda:0")
            raw_output.append(model(sg).detach().cpu())
            parameters.append(params.cpu())
            print(f"{batch+1} / {len(dl)} { dumstring  * 20 }", end="\r")
    model.train()
    output = torch.vstack(raw_output)
    parameters = torch.concat(parameters,dim = 0)
    return combined(output.cpu(), parameters.cpu())

In [7]:
test_model = init_model()
_,_,testds = get_new_train_validation_test_datasets(0.1,0.000001)
testdl = DataLoader(testds, batch_size=1, shuffle=True)
print(len(testdl))
with autograd.detect_anomaly(check_nan=True):
    for batch, (sg,params) in enumerate(testdl):
        out = torch.mean(test_model(sg.to("cuda:0").float().view(1,1,400,400)))
        out.backward()
        print(out)    

507990 length of original set
length of testval set 50800
length of train set 457190
length of test set 50799
length of valid set 1
50799


C:\Users\aashr\AppData\Local\Temp\ipykernel_15588\3674700284.py:5: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with autograd.detect_anomaly(check_nan=True):


tensor(0.4261, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3691, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3007, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.4162, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3237, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5322, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.4294, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(-0.0116, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0821, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1546, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3012, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2521, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0659, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3959, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2753, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2940, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1973, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2940, device='cuda:0'

C:\Users\aashr\AppData\Roaming\Python\Python310\site-packages\torch\autograd\__init__.py:197: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "c:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\aashr\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\aashr\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1041, in launch_instance
    app.start()
  File "C:\Users\aashr\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 724, in start
    self.io_loop.start()
  File "C:\Users\aashr\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start
    self.asyncio_loop.run_f

RuntimeError: Function 'AddmmBackward0' returned nan values in its 2th output.

In [ ]:
list(testds)

In [ ]:

startlr = 3e-5
optimizer = optim.AdamW(params=model.parameters(), lr=startlr)
optimizer1 = optim.NAdam(params=model.parameters(), lr=startlr)
step_scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[1, 2, 3, 4], gamma=0.5
)
# at the end of 600 epochs, the learning rate is 0.000,002,62
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
scheduler_pl = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode="max", factor=0.7, patience=35, verbose=True
)
l1 = nn.L1Loss(reduction="sum")
l2 = nn.MSELoss(reduction="sum")
lossfn = lambda x, y: l1(x, y) + l2(x, y)

In [ ]:
def to_seconds(s):
    return f"{s//3600}H:{(s%3600)//60}M:{round(s%60,3)}S"


def ismult(n, div):
    return bool(1 >> (n % div))

In [ ]:
def save_model(best_model, config, m1, m2, m1name="l1", m2name="l2"):
    try:
        torch.save(
            best_model,
            f"./saved_models/ViT/WithNoise/best_model_state_dict_ViT_regressor_for{config.run_name}_stime_{config.start_time.replace(':', '-')}__{m1name}_{m1}__{m2name}_{m2}.pt",
        )
        print("\nSAVING MODEL")
    except:
        wandb.alert(level="warning", title="OUT OF MEMORY")

In [ ]:
def train_eval_model(config, train_dl, test_dl, adam=True, nadam=False):
    min_mae, min_mse = float("inf"), float("inf")
    ldl = len(train_dl)
    results = pd.DataFrame()
    best_model = OrderedDict()
    for epoch in range(1, config.epochs + 1):
        print("Pre-Evaluation Finished; Starting Training")
        etime = time.time()
        for batch, (sg, params) in enumerate(train_dl):
            stime = time.time()
            sgsh = sg.shape
            sg = sg.to("cuda:0").to(torch.float).view(sgsh[0], 1, sgsh[1], sgsh[2])
            params = params[:, 1:3].to("cuda:0").to(torch.float)
            optimizer.zero_grad()
            outputs = model(sg)
            loss = lossfn(outputs, params)
            loss.backward()
            optimizer.step() if adam else None
            optimizer1.step() if nadam else None
            #
            torch.cuda.empty_cache()
            #
            wandb.log(
                {
                    "loss": loss.item(),
                    "batch_mae": mae(outputs.to("cpu"), params.to("cpu")),
                    "batch_mse": mse(outputs.to("cpu"), params.to("cpu")),
                    "lr": scheduler.get_last_lr()[0],
                    "epoch": epoch,
                }
            )

            print(
                f"{epoch:5}/{config.epochs:5} // {batch:5}/{ldl:5} | Loss: {loss.item():2.4},batch_mae:{mae(outputs.to('cpu'),params.to('cpu')):3.4}, lr:{scheduler.get_last_lr()[0]:1.5}, Time per Batch: {time.time()-stime:.3} seconds, Accumulated Time {to_seconds(round(time.time()-etime,3))}    ",
                end="\r",
                flush=True,
            )

            if (batch - 1) % 5000 == 0:
                epoch_results = calc_metrics(model, test_dl)
                results = pd.concat([results, get_df_from_rdict(epoch_results)])
                min_mae = min(results["MeanAbsoluteError"])
                min_mse = min(results["MeanSquaredError"])
                #
                if epoch_results["MeanAbsoluteError"] == min_mae:
                    best_model = model.state_dict()
                    save_model(
                        best_model,
                        config,
                        list(epoch_results.values())[0],
                        list(epoch_results.values())[1],
                    )

                wandb.log(
                    {"epoch": epoch, "lr": scheduler.get_last_lr()[0]}
                    | epoch_results
                    | {"MinimumMAE": min_mae, "MinimumMSE": min_mse}
                    | {"EpochTime": time.time() - etime}
                )
        #
        epoch_results = calc_metrics(model, test_dl)
        results = pd.concat([results, get_df_from_rdict(epoch_results)])
        #
        min_mae = min(results["MeanAbsoluteError"])
        min_mse = min(results["MeanSquaredError"])
        #
        scheduler.step()
        step_scheduler.step()
        scheduler_pl.step(min_mae)

        if epoch_results["MeanAbsoluteError"] == min_mae:
            best_model = model.state_dict()
            save_model(
                best_model,
                config,
                list(epoch_results.values())[0],
                list(epoch_results.values())[1],
            )
        #

        wandb.log(
            {"epoch": epoch, "lr": scheduler.get_last_lr()[0]}
            | epoch_results
            | {"MinimumMAE": min_mae, "MinimumMSE": min_mse}
            | {"EpochTime": time.time() - etime}
        )

    epoch_results = calc_metrics(model, test_dl)
    results = pd.concat([results, get_df_from_rdict(epoch_results)])
    return min_mae, min_mse

In [ ]:
# uncomment for training
results = []
trials = 1
for i in range(trials):
    wandb.init(
        project="ViT-Regressor-With-Noise",
    )
    config = wandb.config
    config.run_name = wandb.run._run_id
    config = wandb.config
    config.epochs = 3
    config.inx = 400
    config.iny = 400
    config.lr = startlr
    config.trial = i + 1
    config.total_trials = trials
    config.best_model = OrderedDict()
    config.start_time = datetime.datetime.now().isoformat()
    config.savename = f"best_model_state_dict_at_for{config.run_name}_stime_{config.start_time.replace(':', '-')}__acc_max_acc__auc_auc.pt"
    train_dl, valid_dl, test_dl = get_new_train_validation_train_dataloaders()
    train_eval_model(wandb.config, train_dl, valid_dl, nadam=True)
    results.append(calc_metrics(model, test_dl))  # type: ignore
    if i != (trials - 1):
        model = init_model()

In [ ]:
torch.save(
    model.state_dict(),
    f"./saved_models/ViT/WithNoise/best_model_state_dict_ViT_regressor_11_07_2023",
)


In [ ]:
evaldl = test_dl

In [ ]:
model.eval()
totout = []
with torch.no_grad():
    for batch, (sg, params) in enumerate(evaldl):
        sg = sg.to("cuda:0").to(torch.float)
        sgsh = sg.shape
        sg = sg.view(sgsh[0], 1, sgsh[1], sgsh[2])
        modelout = (model(sg).detach().cpu())
        params = params.to("cpu").to(torch.float)
        comb = torch.concat([modelout, params], dim=1)
        print(comb[1],modelout[1],params[1])
        print(batch, "finished")
        totout.append(comb)
model.train()
print(len(totout))

In [ ]:
all_params = torch.cat(totout)

In [ ]:
import pandas as pd

df = pd.DataFrame(all_params.numpy())
df = df.rename(columns={0: "PM1", 1: "PM2", 2: "EOS", 3: "M1", 4: "M2", 5: "SNR",})
df["combined"] = df["M1"] + df["M2"]
df["DiffM1"] = abs(df["M1"] - df["PM1"])
df["DiffM2"] = abs(df["M2"] - df["PM2"])
df["totDiff"] = df["DiffM1"] + df["DiffM2"]
df["avgDiff"] = df["totDiff"] / 2
df.to_csv("results.csv")
df.head()


In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

In [ ]:
diffhist = sns.histplot(df["avgDiff"])
diffhist.set(title = "Difference between predicted and actual masses", xlabel = "Difference (solar masses)", ylabel = "Count")

In [ ]:
sns.regplot(x = df["combined"], y = df["avgDiff"])

In [ ]:
sns.regplot(x=df["M1"], y=df["DiffM1"])


In [ ]:
sns.regplot(x=df["M2"], y=df["DiffM2"])


In [ ]:
df

In [ ]:
temparray = pd.melt(
    df[["SNR", "DiffM2", "DiffM1", "avgDiff"]].rename(columns = {"SNR": "Signal to Noise Ratio", "DiffM2": "Difference between predicted and actual for Mass 2", "DiffM1": "Difference between predicted and actual for Mass 1", "avgDiff": "Average Difference between predicted and actual for both masses", }), id_vars=["Signal to Noise Ratio"])
a = sns.lineplot(data=temparray, x = "Signal to Noise Ratio", y = "value", hue = "variable")
a.set(title = "Difference between predicted and actual masses", xlabel = "Signal to Noise Ratio", ylabel = "Difference between predicted and actual (solar masses)")
new_title = 'Legend:'
# replace labels
plt.legend(title='Legend', loc='upper right')
sns.set(rc={'figure.figsize': (16, 9)})


In [ ]:
a = sns.lineplot(data=pd.melt(df[["combined", "DiffM2", "DiffM1", "avgDiff"]], id_vars=[
                 "combined"]), x="combined", y="value", hue="variable")
a.set(title="Difference between predicted and actual masses",xlabel="Combined Mass (solar masses)", ylabel="Difference between predicted and actual (solar masses)")
sns.set(rc={'figure.figsize': (16, 9)})


In [ ]:
a = sns.violinplot(x=df["EOS"], y=df["avgDiff"])
a.set(title = "Difference between predicted and actual masses vs EOS", xlabel = "Equation of State", ylabel = "Difference between predicted and actual (solar masses)")

In [ ]:
for i in list(dataset.eosmap.items()):
    print(i[0], i[1])

In [ ]:
a,b,c = get_new_test_train_validation_datasets(0,0)

In [ ]:
for i in range(16):
    print(a[i][1])
    plt.imshow(a[i][0].cpu().numpy())
    plt.show()